In [1]:
# Parameters
RUN_DATE = "2025-10-09"


<a href="https://colab.research.google.com/github/HieuNguyenPhi/ADJ_JOBS/blob/main/notebooks/ADJUST_JOB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UAT

In [2]:
import os
from azure.storage.blob import BlobServiceClient

account_name = os.getenv('ACCOUNT_NAME')
account_key = os.getenv('ACCOUNT_KEY')
# Replace with your Azure Storage account name and SAS token or connection string
connect_str = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_list = blob_service_client.list_containers()
container_name = "adjuststbuatprocessed" #os.getenv('CONTAINER_NAME')
container_client = blob_service_client.get_container_client(container_name)
already_processed = [file.name.split('/')[1].split('.')[0] for file in container_client.list_blobs() if file.name.split('/')[0] == 'output']
already_processed[-5:]

['2025-10-04', '2025-10-05', '2025-10-06', '2025-10-07', '2025-10-08']

In [3]:
from datetime import date
import pandas as pd
today = date.today().strftime('%Y-%m-%d')
need_process = pd.date_range(start=already_processed[-1], end=today).strftime('%Y-%m-%d').to_list()
need_process

['2025-10-08', '2025-10-09']

In [4]:
container_name_uat = "adjuststbuat"
container_client_uat = blob_service_client.get_container_client(container_name_uat)
from collections import defaultdict
files = [i.name for i in container_client_uat.list_blobs()]
groups = defaultdict(list)
for f in files:
    dt = f.split('_')[1]
    groups[dt].append(f)
groups[dt]

['rsh20bkkb4zk_2025-10-09T000000_762c775ae454d23f2c6b6a75623d14c7_2853a0.csv.gz',
 'rsh20bkkb4zk_2025-10-09T000000_762c775ae454d23f2c6b6a75623d14c7_2853a1.csv.gz',
 'rsh20bkkb4zk_2025-10-09T000000_762c775ae454d23f2c6b6a75623d14c7_be8220.csv.gz',
 'rsh20bkkb4zk_2025-10-09T000000_762c775ae454d23f2c6b6a75623d14c7_c35750.csv.gz',
 'rsh20bkkb4zk_2025-10-09T000000_762c775ae454d23f2c6b6a75623d14c7_c35751.csv.gz']

In [5]:
import polars as pl 
from tqdm import tqdm
storage_options = {
    "account_name": account_name,
    "account_key":  account_key,
}

for ts, files in tqdm(groups.items()):
    dt = ts[:10]
    if dt not in need_process:
        continue
    df = pl.scan_csv(f"az://adjuststbuat/*_{ts}_*.csv.gz", storage_options = storage_options,glob=True, has_header = True, null_values = ["","NULL"], ignore_errors=True).select(pl.all().cast(pl.Utf8))
    df.sink_parquet(f"az://adjuststbuatprocessed/processing/dt={dt}/{ts}.parquet", storage_options = storage_options, compression="snappy")
    print(f'Done dt={dt}/{ts}.parquet')
        

  0%|          | 0/1858 [00:00<?, ?it/s]

 99%|█████████▊| 1834/1858 [00:10<00:00, 174.06it/s]

Done dt=2025-10-08/2025-10-08T000000.parquet


Done dt=2025-10-08/2025-10-08T010000.parquet


Done dt=2025-10-08/2025-10-08T020000.parquet


...

100%|█████████▉| 1857/1858 [03:20<00:06,  6.10s/it]

Done dt=2025-10-08/2025-10-08T230000.parquet


100%|██████████| 1858/1858 [03:28<00:00,  6.62s/it]

100%|██████████| 1858/1858 [03:28<00:00,  8.90it/s]

Done dt=2025-10-09/2025-10-09T000000.parquet


In [6]:
for dt in need_process:
  df = pl.scan_parquet(f"az://adjuststbuatprocessed/processing/dt={dt}/*.parquet", storage_options=storage_options,glob=True).with_columns(pl.lit(dt).alias("dt"))
  df.sink_parquet(f"az://adjuststbuatprocessed/output/{dt}.parquet", storage_options=storage_options, compression="snappy")
  print(f'\n Done {dt}\n')


 Done 2025-10-08




 Done 2025-10-09



# Live

In [7]:
already_processed = [file.name.split('/')[-1].split('.')[0] for file in container_client.list_blobs() if file.name[:12] == 'live/output/']
already_processed[-5:]

['2025-10-04', '2025-10-05', '2025-10-06', '2025-10-07', '2025-10-08']

In [8]:
need_process = pd.date_range(start=already_processed[-1], end=today).strftime('%Y-%m-%d').to_list()
need_process

['2025-10-08', '2025-10-09']

In [9]:
container_name_uat = "adjuststblive"
container_client_uat = blob_service_client.get_container_client(container_name_uat)
from collections import defaultdict
files = [i.name for i in container_client_uat.list_blobs()]
groups = defaultdict(list)
for f in files:
    dt = f.split('_')[1]
    groups[dt].append(f)
groups[dt]

['65n1fgov4zr4_2025-10-09T000000_762c775ae454d23f2c6b6a75623d14c7_2853a0.csv.gz',
 '65n1fgov4zr4_2025-10-09T000000_762c775ae454d23f2c6b6a75623d14c7_2853a1.csv.gz',
 '65n1fgov4zr4_2025-10-09T000000_762c775ae454d23f2c6b6a75623d14c7_be8220.csv.gz',
 '65n1fgov4zr4_2025-10-09T000000_762c775ae454d23f2c6b6a75623d14c7_be8221.csv.gz',
 '65n1fgov4zr4_2025-10-09T000000_762c775ae454d23f2c6b6a75623d14c7_c35750.csv.gz',
 '65n1fgov4zr4_2025-10-09T000000_762c775ae454d23f2c6b6a75623d14c7_c35751.csv.gz']

In [10]:
storage_options = {
    "account_name": account_name,
    "account_key":  account_key,
}

for ts, files in tqdm(groups.items()):
    dt = ts[:10]
    if dt not in need_process:
        continue
    df = pl.scan_csv(f"az://adjuststblive/*_{ts}_*.csv.gz", storage_options = storage_options,glob=True, has_header = True, null_values = ["","NULL"], ignore_errors=True).select(pl.all().cast(pl.Utf8))
    df.sink_parquet(f"az://adjuststbuatprocessed/live/processing/dt={dt}/{ts}.parquet", storage_options = storage_options, compression="snappy")
    print(f'Done dt={dt}/{ts}.parquet')
        

  0%|          | 0/1839 [00:00<?, ?it/s]

 99%|█████████▊| 1815/1839 [00:25<00:00, 70.06it/s]

Done dt=2025-10-08/2025-10-08T000000.parquet


 99%|█████████▊| 1815/1839 [00:36<00:00, 70.06it/s]

 99%|█████████▊| 1816/1839 [00:49<00:00, 30.51it/s]

...

100%|█████████▉| 1838/1839 [09:54<00:19, 19.88s/it]

Done dt=2025-10-08/2025-10-08T230000.parquet


100%|██████████| 1839/1839 [10:30<00:00, 24.21s/it]

100%|██████████| 1839/1839 [10:30<00:00,  2.92it/s]

Done dt=2025-10-09/2025-10-09T000000.parquet


In [11]:
for dt in need_process:
  df = pl.scan_parquet(f"az://adjuststbuatprocessed/live/processing/dt={dt}/*.parquet", storage_options=storage_options,glob=True).with_columns(pl.lit(dt).alias("dt"))
  df.sink_parquet(f"az://adjuststbuatprocessed/live/output/{dt}.parquet", storage_options=storage_options, compression="snappy")
  print(f'\n Done {dt}\n')


 Done 2025-10-08




 Done 2025-10-09

